In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Згідно завдання необхідно розробити модель передбачення churn rate водія не враховуючи тиждень, тому для початку згрупуємо данні водіїв

In [8]:
from tqdm import tqdm
data = pd.read_csv('../input/techuklon-int20h/train.csv')

def preprocess_df(df):
    if 'target' in df.columns:
        target = ['target']
    else:
        target = []
    train_v = df[df['Week']==0].drop(columns = ['Week'])[['Id']+[f"V{i}" for i in range(1,21)]]
    train_p = df[['Id']+[f'P{i}' for i in range(1,28)]+target].groupby(by=['Id']).agg(list)

    for i in tqdm(range(1,28)):
        for j in [0,1,2,3]:
            train_p[f'P{i}_w{j}'] = train_p[f'P{i}'].apply(lambda x:x[j])
        train_p.drop(columns=[f'P{i}'], inplace=True)
    
    if('target' in df.columns):
        train_p['target'] = train_p['target'].apply(lambda x: max(x))
        
    return train_v.merge(train_p, on=['Id'], how='inner')

In [9]:
data = preprocess_df(data)
data.head()

## Розрахуємо розподіл таргетних классів. Як бачимо, маємо дизбаланс у даних 

In [18]:
data['target'].value_counts()

## Ми помітили, що в даних є досить велика кількість пропусків й ця кількість доволі сильно корелює з таргетною змінною

In [14]:
data["v_nans"] = data[[f'V{i}' for i in range(1,21)]].isna().sum(axis=1)
for j in [0,1,2,3]:
    data[f"p_nans_w{j}"] = data[[f'P{i}_w{j}' for i in range(1,28)]].isna().sum(axis=1)
data[['v_nans']+[f'p_nans_w{j}' for j in range(4)]].corrwith(data['target'])

## Розгянемо які саме параметри найбільш впливаютьна таргетну змінну на прикладі моделі ligthgbm. Для подальшого спрощення аналізу й валідування моделі застосуємо одну з automl бібліотек - pycaret

In [15]:
!pip install pycaret

In [17]:
from pycaret.classification import *
clf1 = setup(data, target = 'target', data_split_stratify = True) 

In [20]:
lightgbm = create_model('lightgbm')
lightgbm = tune_model(lightgbm, optimize = 'AUC')

In [21]:
plot_model(lightgbm, plot = 'feature')


### Так як данні закодовані тяжко робити висносновски стосовно впливу даних на таргетну зміну й застосовувати різноматні комбінації факторів, проте навіть даного аналізу достатньо, щоб отримати фільний скор більше ніж 0.97 